In [11]:
import gspread
import pandas as pd
from gspread_dataframe import get_as_dataframe
from google.oauth2.service_account import Credentials

# Variáveis fixas para o seu projeto
SHEET_ID = "1_OQkQpER2aKNnSqezZb15L9u93i-kn2kBUDBwvjqeQI"
NOME_ABA = "main"
ARQUIVO_CREDENCIAL = "service_account.json"

def carregar_dados_google_sheets(sheet_id: str = SHEET_ID, nome_aba: str = NOME_ABA) -> pd.DataFrame:
    """
    Carrega dados de uma aba específica de um Google Sheets.

    :param sheet_id: ID da planilha no Google Sheets
    :param nome_aba: Nome da aba a ser lida
    :return: DataFrame com os dados lidos
    """
    try:
        # Define os escopos de acesso necessários
        scopes = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        
        # Carrega as credenciais do arquivo de conta de serviço
        credentials = Credentials.from_service_account_file(ARQUIVO_CREDENCIAL, scopes=scopes)
        
        # Autoriza o cliente do gspread
        client = gspread.authorize(credentials)
        
        # Abre a planilha pelo ID
        sheet = client.open_by_key(sheet_id)
        
        # Abre a aba desejada
        worksheet = sheet.worksheet(nome_aba)
        
        # Converte para DataFrame
        df = get_as_dataframe(worksheet, evaluate_formulas=True, dtype=str)
        
        # Remove linhas e colunas totalmente vazias
        #df = df.dropna(how='all').dropna(axis=1, how='all')

        print(f"[INFO] Dados carregados com sucesso! {df.shape[0]} linhas e {df.shape[1]} colunas.")

        return df

    except Exception as e:
        print(f"[ERRO] Falha ao carregar dados do Google Sheets: {str(e)}")
        return pd.DataFrame()  # Retorna DataFrame vazio em caso de erro


In [16]:
CRITERIOS = {
    'ATA': {'V': 7, 'E': 0, 'D': -4, 'GOL': 7, 'ASS': 4, 'STG': 2, 'GC': -10, 'AMA': -4, 'AZUL': -8, 'VER': -16, 'PP': -10, 'GS': 0, 'DD': 0, 'DP': 0},
    'MEI': {'V': 7, 'E': 0, 'D': -4, 'GOL': 8.5, 'ASS': 5, 'STG': 2.5, 'GC': -10, 'AMA': -4, 'AZUL': -8, 'VER': -16, 'PP': -10, 'GS': 0, 'DD': 0, 'DP': 0},
    'ZAG': {'V': 7, 'E': 0, 'D': -4, 'GOL': 10, 'ASS': 6, 'STG': 3, 'GC': -10, 'AMA': -4, 'AZUL': -8, 'VER': -16, 'PP': -10, 'GS': 0, 'DD': 0, 'DP': 0},
    'GK':  {'V': 6, 'E': 0, 'D': -3, 'GOL': 16, 'ASS': 10, 'STG': 4, 'GC': -10, 'AMA': -4, 'AZUL': -8, 'VER': -16, 'PP': -10, 'GS': -5, 'DD': 5, 'DP': 20}
}
def tratar_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Trata o DataFrame: converte tipos e cria a coluna de pontos (PTS).
    """
    # 1. Lista de colunas que devem ser numéricas
    colunas_para_int = ['V', 'E', 'D', 'GOL', 'ASS', 'STG', 'GC', 'AMA', 'AZUL', 'VER', 'PP', 'GS', 'DD', 'DP', 'FALTA']

    # 2. Converte essas colunas para int, preenchendo valores ausentes com 0 antes
    for col in colunas_para_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].fillna(0), errors='coerce').fillna(0).astype(int)

    # 3. Função interna para calcular pontos linha a linha
    def calcular_pontos(row):
        posicao = row['POSIÇÃO']
        if posicao not in CRITERIOS:
            return 0  # posição desconhecida
        pontos = sum(row[col] * CRITERIOS[posicao].get(col, 0) for col in CRITERIOS[posicao])
        return pontos

    # 4. Aplica a função no DataFrame
    df['PTS'] = df.apply(calcular_pontos, axis=1)

    return df

In [19]:
df = carregar_dados_google_sheets()
df = tratar_dataframe(df)

[INFO] Dados carregados com sucesso! 60 linhas e 22 colunas.


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 59
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATA        60 non-null     object 
 1   COMPETIÇÃO  60 non-null     object 
 2   RODADA      60 non-null     object 
 3   PARTIDA     60 non-null     object 
 4   PLAYER      60 non-null     object 
 5   POSIÇÃO     60 non-null     object 
 6   TIME        60 non-null     object 
 7   V           60 non-null     int64  
 8   E           60 non-null     int64  
 9   D           60 non-null     int64  
 10  GOL         60 non-null     int64  
 11  ASS         60 non-null     int64  
 12  STG         60 non-null     int64  
 13  GC          60 non-null     int64  
 14  AMA         60 non-null     int64  
 15  AZUL        60 non-null     int64  
 16  VER         60 non-null     int64  
 17  PP          60 non-null     int64  
 18  GS          60 non-null     int64  
 19  DD          60 non-null     int64  
